# MobileNet fine-tuning on dataset

In [1]:
import os
from pathlib import Path
from typing import Tuple, List, Dict, Any

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt


In [2]:
# Load the .npz file
data_train = np.load('hornet-bees-2/train_frelon.npz', allow_pickle=True)
X_train = data_train['X']
y_train = data_train['y']

data_val = np.load('hornet-bees-2/val_frelon.npz', allow_pickle=True)
X_val = data_val['X'] 
y_val = data_val['y']

data_test = np.load('hornet-bees-2/test_frelon.npz', allow_pickle=True)
X_test = data_test['X']
y_test = data_test['y']

In [3]:
# Preprocess the data
def preprocess_data(X):
    # Normalize the images
    X = X / 255.0
    return X

# Preprocess the datasets
X_train = preprocess_data(X_train)
X_val = preprocess_data(X_val)
X_test = preprocess_data(X_test)

In [5]:
# Convert labels to arrays with consistent shapes
def format_labels(y):
    boxes = [entry['boxes'] for entry in y]  # Keep as a list due to varying shapes
    class_ids = [entry['class_ids'] for entry in y]  # Keep as a list due to varying shapes
    return boxes, class_ids

y_train_boxes, y_train_classes = format_labels(y_train)
y_val_boxes, y_val_classes = format_labels(y_val)
y_test_boxes, y_test_classes = format_labels(y_test)

In [6]:
# Define the model
base_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet', alpha=0.35)
base_model.trainable = False

inputs = tf.keras.layers.Input(shape=(416, 416, 3))
resized_inputs = tf.keras.layers.Resizing(224, 224)(inputs)
normalized_inputs = resized_inputs / 255.0
x = tf.keras.applications.mobilenet_v2.preprocess_input(normalized_inputs)
x = base_model(x, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)

boxes_output = tf.keras.layers.Dense(4, name='boxes')(x)
classes_output = tf.keras.layers.Dense(1, activation='sigmoid', name='class_ids')(x)

model = tf.keras.Model(inputs=inputs, outputs=[boxes_output, classes_output])
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss={
        'boxes': 'mean_squared_error',
        'class_ids': 'binary_crossentropy'
    },
    metrics={
        'boxes': 'mae',
        'class_ids': 'accuracy'
    }
)

In [ ]:
# Pad the labels to ensure consistent lengths
def pad_labels(labels, max_boxes=10):
    padded_boxes = []
    padded_classes = []
    for entry in labels:
        boxes = entry['boxes']
        class_ids = entry['class_ids']
        # Handle empty boxes or class_ids
        if len(boxes) == 0:
            padded_boxes.append(np.zeros((max_boxes, 4), dtype=np.float32))
        else:
            padded_boxes.append(np.pad(boxes, ((0, max_boxes - len(boxes)), (0, 0)), mode='constant') if len(boxes) < max_boxes else boxes[:max_boxes])
        
        if len(class_ids) == 0:
            padded_classes.append(np.zeros((max_boxes,), dtype=np.float32))
        else:
            padded_classes.append(np.pad(class_ids, (0, max_boxes - len(class_ids)), mode='constant') if len(class_ids) < max_boxes else class_ids[:max_boxes])
    return np.array(padded_boxes), np.array(padded_classes)

# Pad the training and validation labels
y_train_boxes_padded, y_train_classes_padded = pad_labels(y_train)
y_val_boxes_padded, y_val_classes_padded = pad_labels(y_val)

# Train the model
history = model.fit(
    X_train,
    {'boxes': y_train_boxes_padded, 'class_ids': y_train_classes_padded},
    validation_data=(X_val, {'boxes': y_val_boxes_padded, 'class_ids': y_val_classes_padded}),
    epochs=10,
    batch_size=32
)

In [ ]:
# Pad the test labels
y_test_boxes_padded, y_test_classes_padded = pad_labels(y_val)

# Evaluate the model
test_loss, test_metrics = model.evaluate(
    X_test,
    {'boxes': y_test_boxes_padded, 'class_ids': y_test_classes_padded}
)

print("Test Loss:", test_loss)
print("Test Metrics:", test_metrics)

ValueError: Data cardinality is ambiguous. Make sure all arrays contain the same number of samples.'x' sizes: 55
'y' sizes: 1, 0, 0, 1, 0, 0, 2, 2, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 2, 0, 1, 2, 1, 1, 1, 2, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 0, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 0, 0, 1, 0, 0, 2, 2, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 2, 0, 1, 2, 1, 1, 1, 2, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 0, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1
